In [7]:
import numpy as np
import math
import ast

In [8]:
f = open("input.txt", "r")
rawstring = f.read()
f.close()

In [9]:
bytes = bytearray.fromhex(rawstring)
bytes_as_bits = ''.join(format(byte, '08b') for byte in bytes)


In [10]:
literals = []

class Packet:
    def __init__(self, version, type_id,src_string) -> None:
        self.version = version
        self.type_id = type_id
        self.operator = self.type_id != 4
        self.src_string = src_string
        self.startSub = []
        self.sub = []

        if self.operator:
            self.l_type_id = int(src_string[6])

            if self.l_type_id == 0:
                self.sub_length = int(src_string[7:7+15], base=2)
                self.startSub.append(7+15)
                self.end = 7+15+self.sub_length
                while self.startSub[-1] != self.end:
                    self.parseSub(self.src_string[self.startSub[-1]:self.end])
                    self.startSub.append(self.startSub[-1]+self.sub[-1].end)
            else: 
                self.n_sub = int(src_string[7:7+11], base=2)
                self.startSub.append(7+11)
                while len(self.sub) != self.n_sub:
                    self.parseSub(self.src_string[self.startSub[-1]:])
                    self.startSub.append(self.startSub[-1]+self.sub[-1].end)
                self.end = self.startSub[-1]
            
        else:
            #literal
            temp = list(self.src_string[6:])
            self.len_lit = 5
            i=0
            while(temp[i] != '0'):
                self.len_lit += 5
                temp.pop(i)
                i += 4
            temp.pop(i)
            temp = temp[:i+4]
            self.literalVal = int("".join(temp),base=2)
            self.end = 6+self.len_lit
            self.val = self.literalVal
            literals.append((f'l{len(literals)}',self.val))

        if self.type_id == 0:
            self.val = sum([x.val for x in self.sub])
            self.ast = ast.Call(func=ast.Name(id='sum', ctx=ast.Load()), args=[ast.List(elts=[x.ast for x in self.sub], ctx=ast.Load())], keywords=[])

        elif self.type_id == 1:
            self.val = np.product([x.val for x in self.sub])
            self.ast = ast.Call(func=ast.Attribute(value=ast.Name(id='math', ctx=ast.Load()), attr='prod', ctx=ast.Load()), args=[ast.List(elts=[x.ast for x in self.sub], ctx=ast.Load())], keywords=[])

        elif self.type_id == 2:
            self.val = min([x.val for x in self.sub])
            self.ast = ast.Call(func=ast.Name(id='min', ctx=ast.Load()), args=[ast.List(elts=[x.ast for x in self.sub], ctx=ast.Load())], keywords=[])
        elif self.type_id == 3:
            self.val = max([x.val for x in self.sub])
            self.ast = ast.Call(func=ast.Name(id='max', ctx=ast.Load()), args=[ast.List(elts=[x.ast for x in self.sub], ctx=ast.Load())], keywords=[])
        elif self.type_id == 4:
            self.ast = ast.Constant(value=self.val)
        elif self.type_id == 5:
            self.val = 1 if self.sub[0].val > self.sub[1].val else 0
            self.ast = ast.IfExp(test=ast.Compare(left=self.sub[0].ast, ops=[ast.Gt()], comparators=[self.sub[1].ast]), body=ast.Constant(value=1), orelse=ast.Constant(value=0))
        elif self.type_id == 6:
            self.val = 1 if self.sub[0].val < self.sub[1].val else 0
            self.ast = ast.IfExp(test=ast.Compare(left=self.sub[0].ast, ops=[ast.Lt()], comparators=[self.sub[1].ast]), body=ast.Constant(value=1), orelse=ast.Constant(value=0))
        elif self.type_id == 7:
            self.val = 1 if self.sub[0].val == self.sub[1].val else 0
            self.ast = ast.IfExp(test=ast.Compare(left=self.sub[0].ast, ops=[ast.Eq()], comparators=[self.sub[1].ast]), body=ast.Constant(value=1), orelse=ast.Constant(value=0))

        

    def summedVersion(self):
        return self.version + sum([x.summedVersion() for x in self.sub])

    def parseSub(self, src_string):
        self.sub.append(inter := Packet.fromString(src_string))

    @staticmethod
    def fromString(src_string):
        version = int(src_string[0:3], base=2)
        type_id = int(src_string[3:6], base=2)
        return Packet(version, type_id, src_string)

In [11]:
outer = Packet.fromString(bytes_as_bits)

In [15]:
print(ast.unparse(ast.fix_missing_locations(module)))

sum([min([max([min([min([min([min([math.prod([max([max([max([min([math.prod([math.prod([min([min([math.prod([max([math.prod([sum([min([1736671104])])])])])])])])])])])])])])])])])])])]), 4, max([522179]), math.prod([19219, 1 if 38942 < 247596226 else 0]), sum([math.prod([7, 8, 14]), math.prod([8, 2, 12]), math.prod([4, 10, 3])]), 131, math.prod([1 if 17052 < 2979 else 0, 46058]), sum([664609]), math.prod([1 if 2159 > 2159 else 0, 580431]), math.prod([13, 200, 84, 116]), 15, math.prod([80]), math.prod([95, 1 if 2 < 16651 else 0]), math.prod([8599772, 1 if sum([10, 14, 2]) < sum([8, 9, 11]) else 0]), 14698, min([6, 176724, 17, 218]), math.prod([1 if 2951 > 61 else 0, 30979330271]), math.prod([1 if sum([5, 2, 6]) > sum([9, 12, 2]) else 0, 35056]), max([21, 8, 10464633, 21766]), math.prod([95, 161]), math.prod([1 if 28 > 28 else 0, 1176227742]), math.prod([102, 1 if sum([2, 4, 11]) > sum([10, 15, 10]) else 0]), min([246, 1554]), math.prod([1 if 1470 == 6827177 else 0, 208]), min([728, 7638

In [12]:
outer.ast
module = ast.Module(body=[ast.Expr(value=outer.ast)],type_ignores=[],lineno=1, col_offset=1)
eval(ast.unparse(ast.fix_missing_locations(module)))

180616437720

In [13]:
outer.val

180616437720

In [20]:
import dis
dis.dis(ast.unparse(ast.fix_missing_locations(module)))

  1           0 LOAD_NAME                0 (sum)
              2 LOAD_NAME                1 (min)
              4 LOAD_NAME                2 (max)
              6 LOAD_NAME                1 (min)
              8 LOAD_NAME                1 (min)
             10 LOAD_NAME                1 (min)
             12 LOAD_NAME                1 (min)
             14 LOAD_NAME                3 (math)
             16 LOAD_METHOD              4 (prod)
             18 LOAD_NAME                2 (max)
             20 LOAD_NAME                2 (max)
             22 LOAD_NAME                2 (max)
             24 LOAD_NAME                1 (min)
             26 LOAD_NAME                3 (math)
             28 LOAD_METHOD              4 (prod)
             30 LOAD_NAME                3 (math)
             32 LOAD_METHOD              4 (prod)
             34 LOAD_NAME                1 (min)
             36 LOAD_NAME                1 (min)
             38 LOAD_NAME                3 (math)
             